<a href="https://colab.research.google.com/github/virf96/Analisis_Texto/blob/master/Categorizacion_malos_gensimv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# librerias

In [ ]:
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import re
import time
from wordcloud import WordCloud
from PIL import Image 

import unicodedata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
%config InlineBackend.figure_formats = ['retina']
import seaborn as sns
import plotly.offline as py
color = sns.color_palette()
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import json
import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

# librerias
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

from emot.emo_unicode import UNICODE_EMO, EMOTICONS
import multiprocessing as mp
from multiprocessing import Pool
pd.options.display.max_colwidth = 200

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn

#import spacy

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

Populating the interactive namespace from numpy and matplotlib


# funciones

In [ ]:
# tokens
## palabras sin acentos ni puntuaciones
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8' , 'ignore')
    return text

## quita caracteres especiales
def remove_special_characters(text, remove_digits=True):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern,' ', text)
    return text

## funcion lista a texto
def list_to_str(list):
    comm = ''
    for words in list: 
        comm = comm + words + ' '
    comm=comm.strip()
    return(comm)

## funcion texto a token por oracion
def tokenize_text(text):
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentences) for sentences in sentences]
    return word_tokens

## funcion que obtiene tokeniza los comentarios
def comments(df):
    split_com=[]
    split_com = [x.split(" ") for x in df['Comentario'].astype(str)]
    return(split_com)

## funcion que obtiene las palabras de los comentarios
def list_to_words(comments):
    lista = []
    for x in comments:
        lista.extend(x)
    return(lista)

## limpia lista de palabras 

## palabras en minuscula sin acentos, caracteres especiales, stopwords
def clean_word(palabras):
    lista=[remove_accented_chars(x.lower()) for x in palabras]  ##lista
    lista=list_to_str(lista) ##string
    lista2=remove_special_characters(lista)  ##string
    word_tokens = nltk.word_tokenize(lista2)
    lista=[x for x in word_tokens if x not in stopwords.words('spanish')]
    return(lista)

## palabras de comentarios en minuscula sin acentos, caracteres especiales, stopwords
stop1=['enero','ene','febrero', 'feb','marzo', 'abril','mayo','junio','agosto','septiembre','octubre','noviembre','dic','diciembre','a',
       'am','ademas','al','algo', 'alguien','algun','alguna','algunas','algunos', 'alguno','alla','alli','ahi','ante','antes','aqui','aquellas','aquello',
       'asi', 'atras','ay',
       'ayer', 'casi','como','con','contra', 'cual', 'cuando', 'de','del','desde','demas','donde','durante','e','el','ella','ellas','ello','ellos',
       'en','entre','esa','esas','ese','eso','esos','esta','estas','este','esto','estos','etc','hasta','hay','la','las','le','les','lo','los','mas','me',
       'mi','mia', 'mias','mio','mios', 'mis','mucho','muchos','muy','nos','nosotras', 'nosotros', 'nuestra', 'nuestras','nuestro','nuestros',
       'o','os','otra', 'otras','otro', 'otros','pm', 'para','pero','poco','por','porque','pues','que','quien','quienes','se','si','sin','sobre','solo','su',
       'sus','suya','suyas','suyo','suyos','tambien', 'tanto','te','ti','tu','tus','tuya','tuyas','tuyo','tuyos','un','una','uno','unos','vosotras',
       'vosotros','vuestra','vuestras','vuestro', 'vuestros', 'y','ya','yo', 'zootopia','zoriana','zulma','zuum',
       'zx','zz','zzzzz', 'ke','xk','vdd']

def clean_com(comments):
    aux=[]
    for x in range(0,len(comments)):
        aux.append([y for y in nltk.word_tokenize(remove_special_characters(comments[x]).strip()) if y not in stop1] )
    return(aux)


#    text = re.sub('\[.*?¿\]\%', ' ', text)
 #   text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
  #  text = re.sub('\w*\d\w*', '', text)

def replaceAccents(word):
        word = word.replace('í','i')
        word = word.replace('ó','o')
        word = word.replace('ò','o')
        word = word.replace('ñ','n')
        word = word.replace('é','e')
        word = word.replace('è','e')
        word = word.replace('á','a')
        word = word.replace('à','a')
        word = word.replace('ü','u')
        word = word.replace('ú','u')
        word = word.replace('ö','o')
        word = word.replace('ë','e')
        word = word.replace('ï','i')
        return word

In [ ]:
# corrector
from collections import Counter

### texto en minisculas
def words(text): 
    return re.findall(r'\w+', text.lower())

## carga de diccionario, viene sin acentos la fuente
#WORDS = Counter(words(open('C:/Users/FerDL/Documents/6 Trabajo/Google Customer Reviews/2 Codigos/corpus_es.txt').read()))
#WORDS = Counter(words(open('C:/Users/cenic/Documents/1 Proyectos/Google Customer Reviews/2 Codigos/corpus_es.txt').read()))
WORDS = Counter(words(open('corpus_es.txt').read()))


def P(word): 
    N=0
    for a in WORDS:
        N=N+WORDS[a]
    "Probability of `word`."
    return WORDS[word] / N

def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

def edits0(word): 
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}

def edits1(word):
    "All edits that are one edit away from `word`."
    alphabet    = 'abcdefghijklmnopqrstuvwxyz'
    "Return all strings that are one edit away from this word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known(edits0(word)) or known(edits1(word)) or known(edits2(word)) or [word])

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def correct_text(text):
    "Correct all the words within a text, returning the corrected text."
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    "Spell-correct word in match, and preserve proper upper/lower/title case."
    word = match.group()
    return case_of(word)(correction(word.lower()))

def case_of(text):
    "Return the case-function appropriate for text: upper, lower, title, or just str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)


In [ ]:
### separar palabras ej cara,gato en cara , gato
def comas2(sentence):
    c=[]
    for word in sentence:
        if word[0]==",":
            word=word[1:]
            continue
        
        a=re.split(',', word)
        c.extend(a)
    sentence=c
    return(sentence)

## separa palabras juntas 
def split_words(lista_principal, lista_separa):
    vocal=["a","e","i","o","u"]
    tres_letras=["aer","air", "ais","aun", "aut","aud","aul", "aur","aug","aum", "aus","aux","eol","eur","euf","euc","oax","oes",
                                             "oia","oir","oid","oig","oib","oim","ois","out","ueb"]
    lista1=lista_principal
    for item in lista_separa:
        lista2=[]
        for word in lista1:
            if word.find(item)==-1:             ### no encuentra la palabra a separar en la palabra en el comentario
                lista2.append(word)
                continue
            elif word[0:len(item)]==word:   ### la palabra a buscar es la misma que la palabra en el comentario
                lista2.append(word)
                continue
                
            elif item[-1]  in vocal and item==word[:len(item)]:   ### la palabra a buscar termina en vocal 
                word1=word[0:len(item)]
                word2=word[len(item):]
                if len(word2)==1:
                    if word2 in ['r','s', 'n', 'd']:
                        lista2.append(word)
                    else:
                        lista2.append(word1)
                    continue
                elif not word2[0] in vocal and word2[1] in vocal: 
                    lista2.append(word1)
                    lista2.append(word2)
                    continue
                elif word2[0] in vocal:
                    lista2.append(word1)
                    lista2.append(word2)
                    continue
                else:
                    lista2.append(word)
                    continue
                    
            elif not item[-1]  in vocal and item==word[:len(item)]:
                word1=word[0:len(item)]
                word2=word[len(item):]
                if item in ["buen", "mejor"] and word in ["buenas", "mejorar"]:
                    lista2.append(word)
                    continue
                elif len(word2)==1:
                    if word2 in vocal:
                        lista2.append(word)
                    else:
                        lista2.append(word1)
                    continue                
                elif not word2[0] in vocal and word2[1] in vocal: 
                    lista2.append(word1)
                    lista2.append(word2)
                    continue
                elif word2[0] in vocal and word2[0:3] in tres_letras:
                    lista2.append(word1)
                    lista2.append(word2)
                    continue
                elif word2[0] in vocal and not word2[1] in vocal:
                    lista2.append(word1)
                    lista2.append(word2)
                    continue
                else: 
                    lista2.append(word)
                    continue
            else:
                lista2.append(word)
        lista1=lista2
    return lista1

#lista_separa2=[]
lista_separa=["articulo", "atencion", "atienden", "banco", "bien", "buen", "bueno", "cajas", "caro", "cliente",
              "comprar", "coppel", "cosa", "credito", "empleados", "estacionamiento", "estan", "falta", "gente", "hacer", "lento", "lugar", "malo",
              "mejor", "mucho", "nada", "pagar", "pago", "pequeno", "persona", "personal", "pesimo", "precio", "ropa", "servicio", "siempre", "solo", "surtido",
              "tienda", "tiene", "tienen", "todo", "variedad"]

In [ ]:
# lematizacion
#encoding: utf8
lemmaDict = {}
#with open('C:/Users/FerDL/Documents/6 Trabajo/Google Customer Reviews/2 Codigos/lemmatization-es.txt', 'rb') as f:
#with open('C:/Users/cenic/Documents/1 Proyectos/Google Customer Reviews/2 Codigos/lemmatization-es.txt', 'rb') as f:
with open('lemmatization-es.txt', 'rb') as f:
    data = f.read().decode('utf8').replace(u'\r', u'').split(u'\n')
    data = [a.split(u'\t') for a in data]

#minusculas=[]
#for i in prueba:
 #   minusculas.append([remove_accented_chars(i[1]), remove_accented_chars(i[0])])
    
for a in data:
    if len(a) >1:
        lemmaDict[remove_accented_chars(a[1])] = remove_accented_chars(a[0])

def lemmatize(word):
    return lemmaDict.get(word, word)

def lematiza_lista(texto):
    split_com=[]
    for sentence in texto:
        split_com.append([lemmatize(word) for word in sentence])
    #texto=split_com
    return(split_com)

In [ ]:
# Function for converting emojis into word
def convert_emojis(text):
    ponctuation = list("[.,]()")
    for emot in UNICODE_EMO:
#        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(","," ").replace(":"," ").split()))
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].split())).replace("::"," ")
    if text[0]==":":
        text=text[1:]
    if text[-1]==":":
        text=text[:len(text)-1]
    text=" ".join(text.split(":")).strip()
    text=text.replace(" .",".").replace(" ,",",")
    return text

## limpia el translate de los comentarios
def limpia_translate(lista):
    aux=[]
    for x in lista:
        if x.find("(translated by google)")==-1:
            aux.append(x)
        elif x[0:22]=="(translated by google)":
            aux.append(x[x.find("(original)")+10:].strip())
        else:
            aux.append(x[:x.find("(translated by google)")].strip())
    return(aux)

In [ ]:
## dictionario de verbos
#pathConjug = "C:\\Users\\FerDL\\Documents\\6 Trabajo\\Moderador\\senti_py_master\\classifier\\data\\dictConjug.json" ## XXX : to fill
#pathConjug = "G:\\Mi unidad\\HomeOffice\\Google Customer Reviews (todos)\\dictConjug.json" ## XXX : to fill
pathConjug = "dictConjug.json" ## XXX : to fill

with open(pathConjug, 'r') as f:
    dictConjug = json.load(f)
    
ponctuation = list("[.,:;!?]()")
  
## verbos en infinitivo
def replaceVerbs(x):
    if len(x)==0:
        x = x
    else:
        addBack = False
        if x[-1] in ponctuation:
            endPonctu = x[-1]
            addBack = True
            x = x[:-1]
        for infinitif in dictConjug.keys():            ## busca en cada verbo
            foundMatch = any(e in x.replace(' ','*') for e in set(dictConjug[infinitif]))    ## busca las conjugaciones
            if foundMatch==False:
                pass
            else:
                x = x.replace(' ','*')
                matches = [e for e in set(dictConjug[infinitif]) if '*'+e+'*' in x] ## reemplaza por el infinitivo
                for e in matches:
                    x = x.replace('*'+e+'*','*'+infinitif+'*')
                del matches
                if x.split('*')[0] in set(dictConjug[infinitif]):
                    x = x.replace(x.split('*')[0]+'*'+x.split('*')[1]+'*'+x.split('*')[2],infinitif+'*'+x.split('*')[1]+'*'+x.split('*')[2])
                if '*'.join(x.split('*')[:2]) in set(dictConjug[infinitif]):
                    x = x.replace('*'.join(x.split('*')[:2])+'*'+x.split('*')[2]+'*'+x.split('*')[3],infinitif+'*'+x.split('*')[2]+'*'+x.split('*')[3])
                if x.split('*')[-1] in set(dictConjug[infinitif]):
                    x = x.replace(x.split('*')[-3]+'*'+x.split('*')[-2]+'*'+x.split('*')[-1],x.split('*')[-3]+'*'+x.split('*')[-2]+'*'+infinitif)
                if '*'.join(x.split('*')[-2:]) in set(dictConjug[infinitif]):
                    x = x.replace(x.split('*')[-4]+'*'+x.split('*')[-3]+'*'+'*'.join(x.split('*')[-2:]),x.split('*')[-4]+'*'+x.split('*')[-3]+'*'+infinitif)
                x = x.replace('*',' ')
        if addBack:
            x = x + endPonctu
        return x

In [ ]:
dict_palabras = {
    "abono": ["abonos"],
    "abandonado": ["abandonadas","abandonada","abandonados"],
    "abierto": ["abierta","abiertas", "abiertos"],
    "accesible": ["accesble","accesibled", "accesibles"],
    "accesorio": ["accesorios"],
    "aceptable": ["aceptables"],
    "acomodado": ["acomodados","acomodada", "acomodadas"],
    "acondicionado": ["acondiccionado","acondicionados","acondicionadoy","acondicuonado", "acondixionado"],
    "acta": ["actas"],
    "actitud": ["actitude", "actitudes", "actitudlas", "actitudle"],
    "actualizar": ["actualizacion","actualizada", "actualizadas", "actualizado","actualizados"],#,"actualizarla","actualizarse"],
    "acuerdo": ["acuerdos"],
    "adecuado": ["adecuados", "adecuadas", "adecuada"],
    "adquirir": ["adquirirla","adquirirlas","adquirirlo","adquirirlos"],
    "afore": ["afores","aforey"],
    "afuera": ["afueras"],
    "agotado": ["agotados", "agotada", "agotadas"],
    "agregar": ["agregarlos"],
    "agradable": ["agradables"],
    "ahora": ["ahorita"],
    "aire": ["aires"],
    "almacen": ["almacenes"],
    "alrededor": ["alrededores"],
    "alto": ["altos"],
    "alta": ["altas"],
    "amabilidad": ["amabilida"],
    "amable": ["amables", "amable", "amablea", "ambles", "amablesal", "amableslos", "ambles", "hamable", "hamables"],
    "amigo": ["amiga","amigas","amigos"],
    "amontonado": ["amontonados", "amontonada", "amontonadas", "amontonadoel", "amontanado"],
    "amplio": ["amplios", "amplioel", "ampli", "amplia", "amplias"],
    "animal": ["animales", "animalitos"],
    "anterior": ["anteriores"],
    "ano": ["anos", "anooos"],
    "apartado": ["apartados"],
    "aplicacion": ["aplicaciones"],
    "app": ["ap"],
    "apuro": ["apuros"],
    "area": ["areas", "areai"],
    "articulo": ["articulos","art","arti","articulls","artuculos", "artuiculos", "item", "stock","producto","productos", "aparato", "aparatos"],
    "arriba": ["aariba"],
    "aspecto": ["aspectos"],
    "asco": ["ascooo", "ascoooooo"],
    "atencion": ["atension","lacatencion", "atemcion", "atecion", "atenccion", "atenciin", "atenciion", "atencin", "atencio", "atenciob","atencionau", "atenciones" ],
    "atender": ["atdnder", "atenderte"],
    "atento": ["atentos"],
    "atienda": ["atiend", "atiendad"],
    "atraso": ["atrasa"],
    "aunque": ["aunq", "auqnue", "auque"],
    "automatico": ["automaticos"],
    "bajo": ["bajos"],
    "bancario": ["bancarios","bancaria", "bancarias"],
    "barato": ["baratos", "barata", "baratas"],
    "bancoppel": ["bamcopel", "banccopel", "bancocopel", "bancocoppel", "bancooel", "bancoopel", "bancoopely", "bancooppel", "bancope", "bancopeel","bancopel", "bancopell", "bancopoel", "bancopol", "bancoppe", "bncoppel"],
    "banco": ["bancos", "bancoy"],
    "bano": ["banos", "banoo", "banosy"],
    "barato": ["baratito", "baratillo"],
    "basico": ["basicos"],
    "bastante": ["bastantes", "bastanes", "bastantito"],
    "bebe": ["bebes"],
    "bien": ["bie", "bieeen", "bieen", "biem"],
    "bodega": ["bodegas"],
    "bolsa": ["bolsas", "bolsita"],
    "bonito": ["bonitas", "bonita", "bonitos"],
    "buen": ["buem", "bue", "bueb", "bueen"],
    "bueno": ["buenos","buena", "buenas", "bueba", "buenaa", "buenad", "buenaza", "bueni", "buenicimo", "buenisima", "buenl", "buenon","buenonisimo", "buenp", "buens"],
    "comprar": ["komprar"],
    "caballero": ["caballeros"],
    "cadena": ["cadenas"],
    "caja": ["cajas", "caj", "cajs" ],
    "cajero": ["csjeros","cajeros", "cajeras", "cajera", "cajaras", "cajers", "cajeta"],
    "calidad": ["calidades", "calida", "calidadesa", "calidas"],
    "calor": ["calores","caloron"],
    "calzado": ["calsado","calzadi", "calzadoen", "calzados", "calzadoy", "calzdo", "calzfo"],
    "cantidad": ["cantidades", "cantidady"],
    "camara": ["camaras"],
    "cambiar": ["cambiarlo","cambiarse","csmbiaran"],
    "campo": ["campos"],
    "cancelado": ["cancelados","cancelada", "canceladas"],
    "capacidad": ["capacidades", "capacitad"],
    "capacitacion": ["capacitaciones","capasitacion"],
    "capacitado": ["capacitadas", "capacitado", "capacitados", "capacittados", "capasitados"],
    "cargo": ["cargos"],
    "carro": ["carros", "auto", "autos", "automovil", "automoviles" ],
    "celular": ["cel","celulares"],
    "cercano": ["cercanos","cercana","cercanas"],
    "cerrado": ["cerrada","cerradas", "cerrados"],
    "chat": ["chad"],
    "claro": ["claros","clara", "claras"],
    "cliente": ["clientes","clienyes"],
    "codigo": ["codigos"],
    "colonia": ["colonias"],
    "complicado": ["complicada"],
    "comprador": ["compradores", "compradora", "compradoras"],
    "comprar": ["comprarla","comprarlas","comprarlo","comprarlos", "comprarle","comprarles"],
    "comprado": ["comprados", "comprada", "compradas"],
    "computadora": ["computadoras","ordenador","computador","pc"],
    "coppel": ["encopel","coopelito"], 
    "correcto": ["correcta"],
    "cosa": ["cosas"],
    "cuanto": ["cuantos"],
    "cuenta": ["cuentas"],
    "de": ["d"],
    "demasiado": ["demasiados","demasiada"],
    "deposito": ["depositos"],
    "descripcion": ["descripciones"],
    "descuento": ["descuentos"],
    "destalle": ["destalles"],
    "dia": ["dias"],
    "diferente": ["diferentes"],
    "dificil": ["dificultad", "dificultades", "dificulta"],
    "direccion": ["direcciones"],
    "disponible": ["disponibles"],
    "doble": ["dobles"],
    "duda": ["dudas"],
    "electrico": ["electricos"],
    "electronico": ["electronicos", "electronica", "electronicas"],
    "elegir": ["elegi", "elegia","elegido"],
    "encontrar": ["encontrarlos","encontro"],
    "entregar": ["entregarlo"],
    "enviar": ["enviarlo", "envio", "envios"],
    "error": ["errores"],
    "existe": ["existen"],
    "existencia": ["existencias"],
    "estacionamiento": ["estacionamiebto","estacionamienro"],
    "facil": ["faciles","facilidad"],
    "falla":["fallando", "fallas"],
    "falta": ["faltan", "faltaron", "falte", "falto"],
    "fecha": ["fechas"],
    "fila": ["filaa"],
    "grosero": ["grosero","groceros","grocera", "grosera", "groseras","grocero","groceras"],
    "groseria": ["groserias","grocerias"],
    "hacer": ["hacerlo","hacerlos", "hacerla", "hacerlas", "hacerle", "hacerles", "hacerme" ],
    "hora": ["horas"],
    "interes": ["intereses"],
    "inventario": ["inventarios"],
    "jaja": ["jajaja","jajajaja","jajajajajaja","jajajajaj", "jajaj","jhajaja","jajajajsj","jajajajaja","jajajajajajjajajajajajajajajajajjajajajajajajajajajajajajqjajajajajajajaajajajjajajajajajajajajajajanajanajajajajajanajajajajajajjajajajajajjajajajajajajajajajajjajajajajajajajajajajajajqjajajajajajajaajajajjajajajajajajajajajajanajanajajajajajanajajajajajajjajajajajajjajajajajajajajajajajjajajajajajajajajajajajajqjajajajajajajaajajajjajajajajajajajajajajanajanajajajajajanajajajajajajjajajajajajjajajajajajajajajajajjajajajajajajajajajajajajqjajajajajajajaajajajjajajajajajajajajajajanajanajajajajajanajajajajajaj","jajs","jajajja","jajajaj","jajjajaj","jajajaaa"],
    "largo": ["largos", "larga", "largas"],
    "lento": ["lentos", "lenta", "lentas", "lentitud"],
    "llegar": ["llegado", "llegada"],
    "lugar": ["lugares","lugarcillo","elugar", "lugat"],
    "malo": ["mala", "malas","malos", "mal","malisisisisisismo"],
    "medida": ["medidas"],
    "mejor": ["mejores", "mejorcito"],
    "mejorar": ["mejorarla","mejorarlas", "mejorarlos", "mejorarlo", "mejorarse"],
    "mensaje": ["mensajes"],
    "mes": ["meses"],
    "mismo": ["misma"],
    "modelo": ["modelos"],
    "mucho": ["muchps","muchas","mucha", "muchos","muchisimo","muchisiimo","muchasimas"],
    "mueble": ["muebles"],
    "muestra": ["muestran"],
    "necesitar": ["nesesita","nesesitas","nesesitan","nesesitaras","nesesito","nesesitaria"],
    "ningun": ["ninguna", "ninguno"],
    "numero": ["numeros", "num"],
    "no": ["ni"],
    "ocasion": ["ocasiones"],
    "oferta": ["ofertan", "ofertas"],
    "opcion": ["opciones"],
    "pagado": ["pagados", "pagadas", "pagada"],
    "pagar": ["pagarlo", "pagarlos","pagarla", "pagarlas", "pagarle", "pagarles"],
    "pagina": ["paginas", "pag"],
    "pago": ["pagos"],
    "pedido": ["pedidos"],
    "persona": ["personas"],
    "pesimo": ["pesima", "pecimo","pecimooo"],
    "poner": ["ponerle", "ponerlo"],
    "postal": ["postales"],
    "precio": ["precios"],
    "primera": ["primer", "primeras", "primero", "primeros"],
    "problema": ["problemas"],
    "proceso": ["procesos"],
    "rapido": ["rapida","rapidamente","rapidez", "rapidos"],
    "recibir": ["recibirlo"],
    "recoger": ["recogi", "recoja"],
    "reembolsar": ["reembolsan", "reembolsaron", "reembolso"],
    "registro": ["registros"],
    "satisfecho": ["satisfecha"],
    "semana": ["semanas"],
    "sencillo": ["sencilla"],
    "servicio": ["servicios", "cervicio", "cervicios","cervisio", "cervico"],
    "senor": ["sr", "senores","sra", "senora", "senoras", "srita" ],
    "sucio": ["sucios", "susio","susios"],
    "talla": ["tallas"],
    "tardado": ["tardados", "tardadas", "tardada"],
    "tarjeta": ["tarjetas"],
    "tener": ["tenerlo", "tienen", "tiene"],
    "tiempo": ["tiempos"],
    "tienda": ["tiendas"],
    "todo": ["todas", "todos","toda"],
    "traba": ["traban", "trabo"],
    "ultimo": ["ultimos","ultimas","ultima"],
    "unico": ["unica"],
    "vario": ["varios","varia", "varias"],
    "ver": ["verlo"],
    "volver": ["volvi", "vuelve"],
    "okay": ["ok"],
    "por": ["x"],
    "porque": ["xq","pq","porq","pork","porke", "x q"],
    "que": ["q","k","qe"], 
    "zapato": ["zaparos","zapatos", "zapatitos", "zapatoz", "zapator","zapatito"]
}

def replace_palabras(word):
    for x,y in dict_palabras.items():
        if word in dict_palabras[x]:
            word==x
            return(x)
    return(word)

In [ ]:
# busca palabras

def busca_palabra(lista, palabra):
    for word in lista:
        if word==palabra:
            return 1
            break
    return 0

def busca_en_lista(lista, palabra):
    j=0
    df=pd.DataFrame(columns=['fila_orig', 'comentario'])
    list_filas=[]
    list_coments=[]
    for item in lista:
        if busca_palabra(item, palabra)==1:
            list_filas.append(j)
            list_coments.append(item)
        j=j+1
    df['fila_orig']=list_filas
    df['comentario']=list_coments
    return df


In [ ]:
# despliega resultados modelos
        
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]) )
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print (documents[doc_index])

# carga informacion

In [ ]:
#df=pd.read_csv('C:\\Users\\FerDL\\Documents\\6 Trabajo\\Redes sociales\\Google Customer Reviews\\DataSetGoogleReviews.csv', encoding='utf-8')
#df=pd.read_csv('G:\\Mi unidad\\HomeOffice\\Google Customer Reviews (todos)\\DataSetGoogleReviews.csv',encoding='utf-8') ##
df=pd.read_csv('DataSetGoogleReviews.csv',encoding='utf-8') ##

df["buen_review"] = df["Estrellas"].apply(lambda x: 1 if x > 3 else 0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596821 entries, 0 to 596820
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name_x            596821 non-null  object
 1   ID                596821 non-null  int64 
 2   TIENDA            596821 non-null  object
 3   Estado            596821 non-null  object
 4   Direccion_goolge  596821 non-null  object
 5   Location          596821 non-null  uint64
 6   name_y            596821 non-null  object
 7   Fecha             596821 non-null  object
 8   Estrellas         596821 non-null  int64 
 9   Nombre            596821 non-null  object
 10  Comentario        596821 non-null  object
 11  FORMATO           596821 non-null  object
 12  Average_google    596821 non-null  int64 
 13  Mujer             596821 non-null  int64 
 14  Hombre            596821 non-null  int64 
 15  Local Guide       596821 non-null  int64 
 16  Opiniones         596821 non-null  int

In [ ]:
#df.loc[ df["ID"]==576 , ['Estado', 'Direccion_goolge','Location']]
#df['Fecha'].head()

In [ ]:
df_com=df[df['Comentario']!='Unspecified']
#df_com=df.drop(subset=['Comentario'])
df_com.head()
df_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186246 entries, 3 to 596818
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   name_x            186246 non-null  object
 1   ID                186246 non-null  int64 
 2   TIENDA            186246 non-null  object
 3   Estado            186246 non-null  object
 4   Direccion_goolge  186246 non-null  object
 5   Location          186246 non-null  uint64
 6   name_y            186246 non-null  object
 7   Fecha             186246 non-null  object
 8   Estrellas         186246 non-null  int64 
 9   Nombre            186246 non-null  object
 10  Comentario        186246 non-null  object
 11  FORMATO           186246 non-null  object
 12  Average_google    186246 non-null  int64 
 13  Mujer             186246 non-null  int64 
 14  Hombre            186246 non-null  int64 
 15  Local Guide       186246 non-null  int64 
 16  Opiniones         186246 non-null  int

In [ ]:
## periodo de la informacion
print(min(df_com['Fecha']),"=>", max(df_com['Fecha']))

In [ ]:
df_com['date2']=pd.to_datetime(df['Fecha']).dt.to_period('D')
df_com['date3']=pd.to_datetime(df['Fecha']).dt.to_period('M')

numero_reviews = df_com.groupby(['date3','buen_review'])['buen_review'].count().unstack().fillna(0)
numero_reviews['total_reviews']=numero_reviews[0]+numero_reviews[1]
numero_reviews['reviews_buenos_perct']=100*numero_reviews[1]/numero_reviews['total_reviews']
numero_reviews['reviews_malos_perct']=100-numero_reviews['reviews_buenos_perct']
numero_reviews.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1104: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/home/euler/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/euler/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



buen_review,0,1,total_reviews,reviews_buenos_perct,reviews_malos_perct
date3,,,,,
2012-06,0.0,1.0,1.0,100.000000,0.000000
2012-07,0.0,1.0,1.0,100.000000,0.000000
2012-08,0.0,1.0,1.0,100.000000,0.000000
2012-09,1.0,4.0,5.0,80.000000,20.000000
2012-11,2.0,1.0,3.0,33.333333,66.666667
2012-12,0.0,1.0,1.0,100.000000,0.000000
2013-01,1.0,2.0,3.0,66.666667,33.333333
2013-02,1.0,1.0,2.0,50.000000,50.000000
2013-03,0.0,4.0,4.0,100.000000,0.000000


In [ ]:
df_freq = numero_reviews.loc[ : ,[0,1] ] #.cumsum();
plt.figure();
df_freq.plot();

In [ ]:
df_freq = numero_reviews.loc[ : ,["reviews_buenos_perct","reviews_malos_perct"] ] #.cumsum();
plt.figure();
df_freq.plot();

In [ ]:
numero_reviews.to_csv("numero_reviews.csv")

In [ ]:
df_com.loc[df_com['date3']=='2012-09', ['date3','buen_review']]

In [ ]:
df_imprime=df_com.loc[: , ["ID", "Estrellas", "buen_review", "date3", "Comentario" ] ]
comment=[convert_emojis(replaceAccents(x.lower().replace("\n", " "))) for x in df_imprime['Comentario'].astype(str)]
comment=limpia_translate(comment)
df_imprime["Comentario2"]=comment
df_imprime.to_csv("Comentarios.csv")

## Se dividen los malos

In [ ]:
df_com2=df_com.loc[df_com['buen_review'] == 0,]
df_com2=df_com2.loc[df_com2['date3']>='2018-01',]
split_2b=[convert_emojis(replaceAccents(x.lower().replace("\n", " "))) for x in df_com2['Comentario'].astype(str)]
del df_com2

pd.DataFrame(split_2b).head()

,0
0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ..."
1,buen lugar para comprar (translated by google) good place to shop
2,le falta surtido en comparacion de otras tiendas de la franquicia. (translated by google) it lacks assortment compared to other stores in the franchise.
3,mucha gente (translated by google) many people
4,mal servicio en atencion en afores (translated by google) poor service in afores care


In [ ]:
split_2b=limpia_translate(split_2b)
split_2b[186:188]

['precios algo elevados para la mala calidad de sus articulos. si sacas a credito te suben demasiado.',
 'buen servicio']

In [ ]:
df_com2=df_com.loc[df_com['buen_review'] == 0, ['ID','Estrellas','date3','Comentario']]
df_com2=df_com2.loc[df_com2['date3']>='2018-01',]
indice=range(0,len(split_2b))
df_com2['indice']=indice
df_com2.set_index('indice')
df_com2['Procesamiento']=split_2b
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ..."
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores


In [ ]:
listCounts = []
big_list = list_to_words(split_2b)
#big_list = [word.decode('utf8', ) for word in big_list]
listCounts = pd.Series(big_list).value_counts()
listCounts.to_csv('palabras_exp_cliente1.csv')

In [ ]:
## limpia comentarios
split_2b=clean_com(split_2b)
split_2b[10:15]

[['pequeno', 'mismas', 'opciones', 'comprar', 'articulos', 'hogar'],
 ['tempatnya', 'nyaman', 'dan', 'ngs'],
 ['tienda',
  'tiene',
  'pocos',
  'productos',
  'malo',
  'es',
  'atencion',
  'cliente',
  'no',
  'es',
  'buena'],
 ['buen', 'servicio'],
 ['buen', 'lugar', 'comprar']]

In [ ]:
## quita comas entre palabras
for i in range(0,len(split_2b)):
    split_2b[i]=comas2(split_2b[i])

split_2b[1:2]

[['buen', 'lugar', 'comprar']]

In [ ]:
list_aux=[]
for x in split_2b:
    list_aux.append(list_to_str(x))

df_com2['Procesamiento2']=list_aux
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento,Procesamiento2
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar,buen lugar comprar
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.,falta surtido comparacion tiendas franquicia
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente,mucha gente
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores,mal servicio atencion afores


In [ ]:
#quita palabras juntas
for i in range(0,len(split_2b)):
    split_2b[i]=split_words(split_2b[i], lista_separa)
    
split_2b[43618:43621]

[]

In [ ]:
list_aux=[]
for x in split_2b:
    list_aux.append(list_to_str(x))

df_com2['Procesamiento3']=list_aux
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento,Procesamiento2,Procesamiento3
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar,buen lugar comprar,buen lugar comprar
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.,falta surtido comparacion tiendas franquicia,falta surtido comparacion tiendas franquicia
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente,mucha gente,mucha gente
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores,mal servicio atencion afores,mal servicio atencion afores


In [ ]:
listCounts = []
big_list = list_to_words(split_2b)
#big_list = [word.decode('utf8', ) for word in big_list]
listCounts = pd.Series(big_list).value_counts()
listCounts.to_csv('palabras_exp_cliente2.csv')

In [ ]:
## reemplaza verbos
split_com=[]
j=0
for i in range(0,len(split_2b)):
    try:
        split_com.append(word_tokenize(replaceVerbs(re.sub(' +', ' ',list_to_str(split_2b[i])))))
    except:
        split_com.append(split_2b[i])
        print("expected string or bytes-like object","=>", i,"=>", j)
        j=j+1
    

expected string or bytes-like object => 40 => 0
expected string or bytes-like object => 49 => 1
expected string or bytes-like object => 79 => 2
expected string or bytes-like object => 162 => 3
expected string or bytes-like object => 287 => 4
expected string or bytes-like object => 301 => 5
expected string or bytes-like object => 315 => 6
expected string or bytes-like object => 515 => 7
expected string or bytes-like object => 523 => 8
expected string or bytes-like object => 560 => 9
expected string or bytes-like object => 565 => 10
expected string or bytes-like object => 606 => 11
expected string or bytes-like object => 618 => 12
expected string or bytes-like object => 630 => 13
expected string or bytes-like object => 650 => 14
expected string or bytes-like object => 692 => 15
expected string or bytes-like object => 770 => 16
expected string or bytes-like object => 791 => 17
expected string or bytes-like object => 822 => 18
expected string or bytes-like object => 844 => 19
expected stri

In [ ]:
list_aux=[]
for x in split_com:
    list_aux.append(list_to_str(x))

df_com2['Procesamiento4']=split_com
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento,Procesamiento2,Procesamiento3,Procesamiento4
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,"[tienda, tener, variedad, articulos, tal, vez, no, todos, gustos, tamano, supongo, tener, diferentes, opciones, gustos, gente, malo, ser, no, tener, amplio, estacionamiento, unicos, cajones, tener..."
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar,buen lugar comprar,buen lugar comprar,"[buen, lugar, comprar]"
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.,falta surtido comparacion tiendas franquicia,falta surtido comparacion tiendas franquicia,"[falta, surtido, comparacion, tiendas, franquicia]"
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente,mucha gente,mucha gente,"[mucha, gente]"
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores,mal servicio atencion afores,mal servicio atencion afores,"[mal, servicio, atencion, afores]"


In [ ]:
#nltk.download('stopwords')
stop2=stopwords.words('spanish')
stop2.extend(['tener','ser','estar'])
I=['nada','todo','todos','no','pero','muy','ni','mucho','muchos','poco']

stop2=[word for word in stop2 if not word in I]

stop2

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'nos',
 'durante',
 'uno',
 'les',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'quienes',
 'cual',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro',
 'vuestra',
 'vuestros',
 'vuestras',
 'esos',
 'esas',
 'estoy',
 'estás',
 'está',
 '

In [ ]:
## dictionario creado
for i in range(0,len(split_com)):
    split_com[i]=[replace_palabras(word) for word in split_com[i] if word not in stop2]

split_com[10]

['pequeno', 'mismas', 'opcion', 'comprar', 'articulo', 'hogar']

In [ ]:
list_aux=[]
for x in split_com:
    list_aux.append(list_to_str(x))

df_com2['Procesamiento5']=split_com
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento,Procesamiento2,Procesamiento3,Procesamiento4,Procesamiento5
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,"[tienda, tener, variedad, articulos, tal, vez, no, todos, gustos, tamano, supongo, tener, diferentes, opciones, gustos, gente, malo, ser, no, tener, amplio, estacionamiento, unicos, cajones, tener...","[tienda, variedad, articulo, tal, vez, no, todo, gustos, tamano, supongo, diferente, opcion, gustos, gente, malo, no, amplio, estacionamiento, unicos, cajones, no, bien, aprovechados, motos, ocupa..."
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar,buen lugar comprar,buen lugar comprar,"[buen, lugar, comprar]","[buen, lugar, comprar]"
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.,falta surtido comparacion tiendas franquicia,falta surtido comparacion tiendas franquicia,"[falta, surtido, comparacion, tiendas, franquicia]","[falta, surtido, comparacion, tienda, franquicia]"
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente,mucha gente,mucha gente,"[mucha, gente]","[mucho, gente]"
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores,mal servicio atencion afores,mal servicio atencion afores,"[mal, servicio, atencion, afores]","[malo, servicio, atencion, afore]"


In [ ]:
listCounts = []
big_list = list_to_words(split_2b)
#big_list = [word.decode('utf8', ) for word in big_list]
listCounts = pd.Series(big_list).value_counts()
listCounts.to_csv('palabras_exp_cliente3.csv')

In [ ]:
## segundo stop
for i in range(0,len(split_com)):
    split_com[i]=[word for word in split_com[i] if word not in stop2 and len(word)>1]
    
split_com[10]

['pequeno', 'mismas', 'opcion', 'comprar', 'articulo', 'hogar']

In [ ]:
listCounts = []
big_list = list_to_words(split_2b)
#big_list = [word.decode('utf8', ) for word in big_list]
listCounts = pd.Series(big_list).value_counts()
listCounts.to_csv('palabras_exp_cliente3.csv')

In [ ]:
df_com2.loc[df_com2['indice']== ,]

In [ ]:
prueba2=busca_en_lista(split_com, "dos")

In [ ]:
## lematiza
split_com3=lematiza_lista(split_com)

In [ ]:
def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list

stops=Union(stop1, stop2)

## tercer stop y diccionario
for i in range(0,len(split_com3)):
    split_com3[i]=[replace_palabras(word) for word in split_com3[i] if not word in stops]

split_com3[10]

['pequeno', 'mismo', 'opcion', 'comprar', 'articulo', 'hogar']

In [ ]:
list_aux=[]
for x in split_com3:
    list_aux.append(list_to_str(x))
    
df_com2['Procesamiento6']=list_aux
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento,Procesamiento2,Procesamiento3,Procesamiento4,Procesamiento5,Procesamiento6
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,"[tienda, tener, variedad, articulos, tal, vez, no, todos, gustos, tamano, supongo, tener, diferentes, opciones, gustos, gente, malo, ser, no, tener, amplio, estacionamiento, unicos, cajones, tener...","[tienda, variedad, articulo, tal, vez, no, todo, gustos, tamano, supongo, diferente, opcion, gustos, gente, malo, no, amplio, estacionamiento, unicos, cajones, no, bien, aprovechados, motos, ocupa...",tienda variedad articulo tal vez no todo gusto tamano suponer diferente opcion gusto gente malo no ampliar estacionamiento unico cajon no bien aprovechar moto ocupar espacio orden estacionar moto ...
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar,buen lugar comprar,buen lugar comprar,"[buen, lugar, comprar]","[buen, lugar, comprar]",bueno lugar comprar
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.,falta surtido comparacion tiendas franquicia,falta surtido comparacion tiendas franquicia,"[falta, surtido, comparacion, tiendas, franquicia]","[falta, surtido, comparacion, tienda, franquicia]",falta surtir comparacion tienda franquicia
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente,mucha gente,mucha gente,"[mucha, gente]","[mucho, gente]",gente
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores,mal servicio atencion afores,mal servicio atencion afores,"[mal, servicio, atencion, afores]","[malo, servicio, atencion, afore]",malo servicio atencion afore


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(split_com3, min_count=5, threshold=30) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[split_com3], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[split_com3[0]]])

['tienda', 'variedad', 'articulo', 'tal_vez', 'no', 'todo', 'gusto', 'tamano', 'suponer', 'diferente', 'opcion', 'gusto', 'gente', 'malo', 'no', 'ampliar', 'estacionamiento', 'unico', 'cajon', 'no', 'bien', 'aprovechar', 'moto', 'ocupar', 'espacio', 'orden', 'estacionar', 'moto', 'taxi', 'lado', 'ocupar', 'tapar', 'entrar', 'mismo', 'cajon', 'estacionamiento', 'ende', 'no', 'permitir', 'entrar', 'salir', 'librar', 'vehiculo', 'moto', 'concideracion', 'momento', 'visitar', 'tienda']


In [ ]:
 list(bigram[split_com3])

[['tienda',
  'variedad',
  'articulo',
  'tal_vez',
  'no',
  'todo',
  'gusto',
  'tamano',
  'suponer',
  'diferente',
  'opcion',
  'gusto',
  'gente',
  'malo',
  'no',
  'ampliar',
  'estacionamiento',
  'unico',
  'cajon',
  'no',
  'bien',
  'aprovechar',
  'moto',
  'ocupar',
  'espacio',
  'orden',
  'estacionar',
  'moto',
  'taxi',
  'lado',
  'ocupar',
  'tapar',
  'entrar',
  'mismo',
  'cajon',
  'estacionamiento',
  'ende',
  'no',
  'permitir',
  'entrar',
  'salir',
  'librar',
  'vehiculo',
  'moto',
  'concideracion',
  'momento',
  'visitar',
  'tienda'],
 ['bueno', 'lugar', 'comprar'],
 ['falta', 'surtir', 'comparacion', 'tienda', 'franquicia'],
 ['gente'],
 ['malo', 'servicio', 'atencion', 'afore'],
 ['nada', 'malo'],
 ['xx'],
 ['malo', 'servicio'],
 ['veenden', 'zapato', 'bajar', 'calidad', 'caro', 'calidad', 'desechable'],
 ['variedad',
  'limitar',
  'falso',
  'promocionar',
  'ofertar',
  'articulo',
  'llegar',
  'no',
  'siempre',
  'alegar',
  'terminar']

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
#data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(split_com3)

data_words_bigrams[0:10]

[['tienda',
  'variedad',
  'articulo',
  'tal_vez',
  'no',
  'todo',
  'gusto',
  'tamano',
  'suponer',
  'diferente',
  'opcion',
  'gusto',
  'gente',
  'malo',
  'no',
  'ampliar',
  'estacionamiento',
  'unico',
  'cajon',
  'no',
  'bien',
  'aprovechar',
  'moto',
  'ocupar',
  'espacio',
  'orden',
  'estacionar',
  'moto',
  'taxi',
  'lado',
  'ocupar',
  'tapar',
  'entrar',
  'mismo',
  'cajon',
  'estacionamiento',
  'ende',
  'no',
  'permitir',
  'entrar',
  'salir',
  'librar',
  'vehiculo',
  'moto',
  'concideracion',
  'momento',
  'visitar',
  'tienda'],
 ['bueno', 'lugar', 'comprar'],
 ['falta', 'surtir', 'comparacion', 'tienda', 'franquicia'],
 ['gente'],
 ['malo', 'servicio', 'atencion', 'afore'],
 ['nada', 'malo'],
 ['xx'],
 ['malo', 'servicio'],
 ['veenden', 'zapato', 'bajar', 'calidad', 'caro', 'calidad', 'desechable'],
 ['variedad',
  'limitar',
  'falso',
  'promocionar',
  'ofertar',
  'articulo',
  'llegar',
  'no',
  'siempre',
  'alegar',
  'terminar']

In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 4), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ampliar', 1),
  ('aprovechar', 1),
  ('articulo', 1),
  ('bien', 1),
  ('cajon', 2),
  ('concideracion', 1),
  ('diferente', 1),
  ('ende', 1),
  ('entrar', 2),
  ('espacio', 1),
  ('estacionamiento', 2),
  ('estacionar', 1),
  ('gente', 1),
  ('gusto', 2),
  ('lado', 1),
  ('librar', 1),
  ('malo', 1),
  ('mismo', 1),
  ('momento', 1),
  ('moto', 3),
  ('no', 4),
  ('ocupar', 2),
  ('opcion', 1),
  ('orden', 1),
  ('permitir', 1),
  ('salir', 1),
  ('suponer', 1),
  ('tal_vez', 1),
  ('tamano', 1),
  ('tapar', 1),
  ('taxi', 1),
  ('tienda', 2),
  ('todo', 1),
  ('unico', 1),
  ('variedad', 1),
  ('vehiculo', 1),
  ('visitar', 1)]]

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=2000,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=1000,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [ ]:
start = time.time()
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()# total time taken
print(f"Runtime of the program is {end - start}")

100%|██████████| 540/540 [3:05:37<00:00, 20.63s/it]  

Runtime of the program is -149.25164079666138


In [ ]:
pd.DataFrame(model_results)

,Validation_Set,Topics,Alpha,Beta,Coherence
0,75% Corpus,2,0.01,0.01,0.439495
1,75% Corpus,2,0.01,0.31,0.435480
2,75% Corpus,2,0.01,0.61,0.455003
3,75% Corpus,2,0.01,0.91,0.455003
4,75% Corpus,2,0.01,symmetric,0.455003
...,...,...,...,...,...
535,100% Corpus,10,asymmetric,0.01,0.541931
536,100% Corpus,10,asymmetric,0.31,0.505944
537,100% Corpus,10,asymmetric,0.61,0.500074
538,100% Corpus,10,asymmetric,0.91,0.496906


In [ ]:
## best model

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           chunksize=1000,
                                           passes=10,
                                           alpha='asymmetric',
                                           eta=0.61,
                                           per_word_topics=True)

In [ ]:
lda_model.print_topics(6, num_words=25)[:25]

[(0,
  '0.051*"no" + 0.049*"bueno" + 0.026*"atencion" + 0.020*"servicio" + 0.019*"articulo" + 0.019*"tienda" + 0.019*"todo" + 0.019*"bien" + 0.018*"lugar" + 0.017*"falta" + 0.017*"persona" + 0.016*"variedad" + 0.015*"malo" + 0.011*"atender" + 0.010*"preciar" + 0.010*"surtir" + 0.010*"caro" + 0.009*"caja" + 0.009*"gente" + 0.009*"ropa" + 0.009*"hacer" + 0.008*"coppel" + 0.008*"pequeno" + 0.008*"estacionamiento" + 0.007*"lento"'),
 (1,
  '0.060*"no" + 0.057*"malo" + 0.029*"servicio" + 0.021*"ir" + 0.017*"hacer" + 0.016*"atencion" + 0.015*"atender" + 0.015*"comprar" + 0.013*"persona" + 0.013*"cliente" + 0.011*"empleado" + 0.009*"poder" + 0.009*"pagar" + 0.008*"decir" + 0.008*"bien" + 0.008*"cajero" + 0.008*"caja" + 0.008*"querer" + 0.007*"buscar" + 0.007*"tienda" + 0.007*"haber" + 0.006*"coppel" + 0.005*"tardar" + 0.005*"lugar" + 0.005*"abrir"'),
 (2,
  '0.020*"siempre" + 0.017*"malo" + 0.017*"cosa" + 0.008*"falta" + 0.008*"remodelacion" + 0.007*"sucio" + 0.007*"mejor" + 0.007*"caja" + 0.

In [ ]:
topics=pd.DataFrame(lda_model.get_document_topics(corpus))
#topics[0]
topics.head()

,0,1,2,3,4,5
0,"(0, 0.64918655)","(1, 0.05762409)","(4, 0.28522724)",None,None,None
1,"(0, 0.8210103)","(1, 0.053866636)","(2, 0.040392533)","(3, 0.03290276)","(4, 0.02778204)","(5, 0.024045777)"
2,"(0, 0.88165015)","(1, 0.034896564)","(2, 0.02693349)","(3, 0.021943057)","(4, 0.018536197)","(5, 0.016040588)"
3,"(0, 0.64506406)","(1, 0.10486583)","(2, 0.080659054)","(3, 0.06576302)","(4, 0.055556722)","(5, 0.048091292)"
4,"(0, 0.06483874)","(1, 0.83499545)","(2, 0.032358576)","(3, 0.026345251)","(4, 0.022224864)","(5, 0.019237163)"


In [ ]:
topic_vector_topic0 = [ x[0] for x in lda_model.get_document_topics(corpus , per_word_topics=False)]
topic_vector_topic1 = [ x[1] for x in lda_model.get_document_topics(corpus , per_word_topics=False)]
topic_vector_topic2 = [ x[2] for x in lda_model.get_document_topics(corpus , per_word_topics=False)]
topic_vector_topic3 = [ x[3] for x in lda_model.get_document_topics(corpus , per_word_topics=False)]
topic_vector_topic4 = [ x[4] for x in lda_model.get_document_topics(corpus , per_word_topics=False)]
topic_vector_topic5 = [ x[5] for x in lda_model.get_document_topics(corpus , per_word_topics=False)]

In [ ]:
def get_topic_especif(lista):
    list_aux=[]
    for i in range(0,len(lista)):
        list_aux.append(lista[i][1])
    return list_aux

In [ ]:
topic_vector_topic0=get_topic_especif(topic_vector_topic0)
topic_vector_topic1=get_topic_especif(topic_vector_topic1)
topic_vector_topic2=get_topic_especif(topic_vector_topic2)
topic_vector_topic3=get_topic_especif(topic_vector_topic3)
topic_vector_topic4=get_topic_especif(topic_vector_topic4)
topic_vector_topic5=get_topic_especif(topic_vector_topic5)



In [ ]:
topics={'Topic1':topic_vector_topic0,
        'Topic2':topic_vector_topic1,
        'Topic3':topic_vector_topic2,
        'Topic4':topic_vector_topic3,
        'Topic5':topic_vector_topic4,
        'Topic6':topic_vector_topic5
       }
topics=pd.DataFrame(topics)
topics.head()

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6
0,0.653711,0.035252,0.003308,0.002689,0.289843,0.001964
1,0.821057,0.053880,0.040393,0.032903,0.027782,0.024046
2,0.881650,0.034897,0.026934,0.021943,0.018536,0.016041
3,0.645077,0.104829,0.080659,0.065763,0.055557,0.048091
4,0.064914,0.834909,0.032359,0.026345,0.022225,0.019237


In [ ]:
topics['max_topics']=topics.idxmax(axis=1)
topics.index=df_com2.index
df_com2_impr=df_com2.loc[:,['ID','date3','Estrellas','Procesamiento', 'Procesamiento3','Procesamiento5','Procesamiento6']]
df_com2_impr=pd.concat([df_com2_impr, topics], axis=1)
df_com2_impr.to_csv("Topic_gensim.csv")

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.157108 -0.056787       1        1  41.009804
1      0.132867  0.049938       2        1  39.945675
4      0.008005  0.047647       3        1   7.685480
3     -0.019534  0.014065       4        1   4.284667
2     -0.049978 -0.058012       5        1   4.160860
5     -0.228468  0.003149       6        1   2.913509, topic_info=        Term          Freq         Total Category  logprob  loglift
20        no  12621.000000  12621.000000  Default  30.0000  30.0000
37     bueno   5548.000000   5548.000000  Default  29.0000  29.0000
16      malo   7484.000000   7484.000000  Default  28.0000  28.0000
61   siempre   1154.000000   1154.000000  Default  27.0000  27.0000
75        ir   2969.000000   2969.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
445       my     11.682066     20.002476   Topic6  -6.4350   2.9980
863       on     15.792208     31.824549   Topic6  -6.1335   2.8351
758     nice     12.885438     24.292934   Topic6  -6.3369   2.9017
773       at     13.469875     37.848347   Topic6  -6.2926   2.5027
20        no     21.620018  12621.161133   Topic6  -5.8194  -2.8337

[450 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2631      1  0.107737  _closed_eyes
2631      2  0.107737  _closed_eyes
2631      4  0.646419  _closed_eyes
2631      5  0.107737  _closed_eyes
292       1  0.253003         abono
...     ...       ...           ...
54        2  0.306830        zapato
54        3  0.163144        zapato
54        4  0.017961        zapato
54        5  0.046399        zapato
54        6  0.001497        zapato

[1642 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 4, 3, 6])

In [ ]:
listCounts = []
big_list = list_to_words(split_com3)
#big_list = [word.decode('utf8', ) for word in big_list]
listCounts = pd.Series(big_list).value_counts()
#listCounts.to_csv('palabras_exp_cliente6.csv')
listCounts.to_csv('palabras_exp_lemmatization.csv')

In [ ]:
## corrige texto
#for x in range(0,len(split_com3)):
#    for word in range(0,len(split_com3[x])):
#        split_com3[x][word]=correct_text(split_com3[x][word])

#split_com3[10:13]

In [ ]:
#listCounts = []
#big_list = list_to_words(split_com3)
##big_list = [word.decode('utf8', ) for word in big_list]
#listCounts = pd.Series(big_list).value_counts()
#listCounts.to_csv('palabras_exp_cliente5.csv')

## Paralelizacion

In [ ]:
num_partitions = 80 #number of partitions to split dataframe
num_cores = 80 #number of cores on your machine

In [ ]:
#funcion que paraleliza
#necesita una serie de pandas para poder concatenar
#resultado regresa una serie pandas
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

#funcion a paralelizar
## corrige texto
def corrige_texto(lista):
    list_aux=lista
    for x in range(0,len(list_aux)):
        for word in range(0,len(list_aux[x])):
            list_aux[x][word]=correct_text(list_aux[x][word])
    return pd.Series(list_aux)


In [ ]:
start=time.time()
split_corrige=parallelize_dataframe(split_com3, corrige_texto)
end=time.time()
# total time taken
print(f"Runtime of the program is {end - start}")
split_corrige[0:3]

Runtime of the program is 816.6586191654205


0    [tienda, variedad, articulo, tal, vez, todo, gusto, tamano, suponer, diferente, opcion, gusto, gente, malo, ampliar, estacionamiento, unico, canon, bien, aprovechar, moto, ocupar, espacio, orden, ...
1                                                                                                                                                                                    [bueno, lugar, comprar]
2                                                                                                                                                           [falto, surtir, comparacion, tienda, franquicia]
dtype: object

In [ ]:
list_aux=[]
for x in split_corrige:
    list_aux.append(list_to_str(x))
    
df_com2['Procesamiento7']=list_aux
df_com2.head()

,ID,Estrellas,date3,Comentario,indice,Procesamiento,Procesamiento2,Procesamiento3,Procesamiento4,Procesamiento5,Procesamiento6,Procesamiento7
11,1256,3,2020-03,"La tienda en si tiene una variedad de artículos, tal vez no tanto como para todos los gustos por el tamaño supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",0,"la tienda en si tiene una variedad de articulos, tal vez no tanto como para todos los gustos por el tamano supongo pero si tiene diferentes opciones para los gustos de la gente, lo malo es que no ...",tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,tienda tiene variedad articulos tal vez no todos gustos tamano supongo tiene diferentes opciones gustos gente malo es no tiene amplio estacionamiento unicos cajones tiene no son bien aprovechados ...,"[tienda, tener, variedad, articulos, tal, vez, no, todos, gustos, tamano, supongo, tener, diferentes, opciones, gustos, gente, malo, ser, no, tener, amplio, estacionamiento, unicos, cajones, tener...","[tienda, variedad, articulo, tal, vez, todo, gustos, tamano, supongo, diferente, opcion, gustos, gente, malo, amplio, estacionamiento, unicos, cajones, bien, aprovechados, motos, ocupan, espacios,...",tienda variedad articulo tal vez todo gusto tamano suponer diferente opcion gusto gente malo ampliar estacionamiento unico cajon bien aprovechar moto ocupar espacio orden estacionar moto taxi lado...,tienda variedad articulo tal vez todo gusto tamano suponer diferente opcion gusto gente malo ampliar estacionamiento unico canon bien aprovechar moto ocupar espacio orden estacionar moto taxi lado...
30,1256,3,2020-02,Buen lugar para comprar\n\n(Translated by Google)\nGood place to shop,1,buen lugar para comprar,buen lugar comprar,buen lugar comprar,"[buen, lugar, comprar]","[buen, lugar, comprar]",bueno lugar comprar,bueno lugar comprar
52,1256,3,2020-01,Le falta surtido en comparación de otras tiendas de la franquicia.\n\n(Translated by Google)\nIt lacks assortment compared to other stores in the franchise.,2,le falta surtido en comparacion de otras tiendas de la franquicia.,falta surtido comparacion tiendas franquicia,falta surtido comparacion tiendas franquicia,"[falta, surtido, comparacion, tiendas, franquicia]","[falta, surtido, comparacion, tienda, franquicia]",falto surtir comparacion tienda franquicia,falto surtir comparacion tienda franquicia
63,1256,3,2020-01,Mucha gente\n\n(Translated by Google)\nMany people,3,mucha gente,mucha gente,mucha gente,"[mucha, gente]","[mucho, gente]",gente,gente
69,1256,2,2020-01,Mal servicio en atención en afores\n\n(Translated by Google)\nPoor service in Afores care,4,mal servicio en atencion en afores,mal servicio atencion afores,mal servicio atencion afores,"[mal, servicio, atencion, afores]","[malo, servicio, atencion, afore]",malo servicio atencion aforar,malo servicio atencion aflorar


In [ ]:
listCounts = []
big_list = list_to_words(split_corrige)
#big_list = [word.decode('utf8', ) for word in big_list]
listCounts = pd.Series(big_list).value_counts()
listCounts.to_csv('palabras_exp_cliente7.csv')

In [ ]:
df_com2.loc[df_com2['indice']==91,]

,Comentario,indice,Procesamiento,Procesamiento2,Procesamiento3,Procesamiento4,Procesamiento5,Procesamiento6,Procesamiento7
957,Solo les falta Bender roa\n\n(Translated by Google)\nThey just need Bender roa,91,solo les falta bender roa,solo falta bender roa,solo falta bender roa,"[solo, falta, bender, roa]","[solo, falta, bender, roa]",solo falto bender roer,solo falto bender roed


In [ ]:
### lista para comenzar el procesamiento del modelo
list_aux=[]
for x in split_corrige:
    list_aux.append(x)

list_aux[0:3]

[['tienda',
  'variedad',
  'articulo',
  'tal',
  'vez',
  'todo',
  'gusto',
  'tamano',
  'suponer',
  'diferente',
  'opcion',
  'gusto',
  'gente',
  'malo',
  'ampliar',
  'estacionamiento',
  'unico',
  'canon',
  'bien',
  'aprovechar',
  'moto',
  'ocupar',
  'espacio',
  'orden',
  'estacionar',
  'moto',
  'taxi',
  'lado',
  'ocupar',
  'tapar',
  'entrar',
  'mismo',
  'canon',
  'estacionamiento',
  'ende',
  'permitir',
  'entrar',
  'salir',
  'librar',
  'vehiculo',
  'moto',
  'consideracion',
  'momento',
  'visitar',
  'tienda'],
 ['bueno', 'lugar', 'comprar'],
 ['falto', 'surtir', 'comparacion', 'tienda', 'franquicia']]

In [ ]:
# list_aux contiene texto limpio y lematizacion sin correccion ortografica
comentarios=pd.Series([data_words_bigrams.index])
for x in range(0,len(data_words_bigrams)):
    comentarios[x]=list_to_str(data_words_bigrams[x])

comentarios[0:2]

0    tienda variedad articulo tal_vez no todo gusto tamano suponer diferente opcion gusto gente malo no ampliar estacionamiento unico cajon no bien aprovechar moto ocupar espacio orden estacionar moto ...
1                                                                                                                                                                                        bueno lugar comprar
dtype: object

In [ ]:
# list_aux contiene texto limpio y lematizacion con correcion ortografica
comentarios=pd.Series([list_aux.index])
for x in range(0,len(list_aux)):
    comentarios[x]=list_to_str(list_aux[x])

comentarios[0:2]